In [1]:
# Import necessary libraries
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as plticker
import math
import cv2
import copy
import random
from collections import deque

In [2]:
# To show image in new window
%matplotlib qt

In [3]:
# Define starting and goal positions
startX, startY = 960, 280
goalX, goalY =  580, 400

# Thresholding distance for node
thresh = 5

# Range for which robot can automatically detect goal
range = 5

# Number of iterations
numIterations = 1000

# Workspace Figure settings
fig = plt.figure()                                                               
ax = fig.add_subplot(1,1,1) 

# Loading Workspace Image
workspace = cv2.imread('workspace_2.png',0)
workspace= workspace[::,::]

# Thresholding the image to remove possible noisy cells
_, workspace = cv2.threshold(workspace, 127, 255, cv2.THRESH_BINARY)

workspaceplot = ax.imshow(workspace)

ax.scatter(startX, startY,marker='x',color='red')
ax.scatter(goalX, goalY, marker='o', color='blue')
ax.axis([0, 1500, 0, 1500])


[0, 1500, 0, 1500]

In [4]:
class node():
    def __init__(self, xnode, ynode):
        self.x = xnode
        self.y = ynode
        self.theta = 0 # Heading angle
        
        self.parentx = 0
        self.parenty = 0
        
    def get_parent(self):
        return self.x, self.y
    def set_parent(self, xval, yval):
        self.x, self.y = xval, yval
    
    def get_heading(self):
        self.theta = math.atan2((self.parenty-self.y),(self.parentx-self.x)) # this is in radians
        return self.theta
    def set_heading(self, thetaval):
        self.theta = thetaval
        
    def isValid(self):
        if(self.x>0 and self.x<1500 and self.y>0 and self.y<1500):
            return True
        else:
            return False

    def isObstacle(self):   # Tolerance of one cell with obstacle
        x = int(self.x)
        y = int(self.y)
        if(workspace[y,x]==0):
            return True
        else:
            return False
        
#     def isDestination(self):
#         if(self.x==goalX and self.y==goalY):
#             return True
#         else:
#             return False  
        

In [5]:
def isCollision(node1, node2):
    # Equation of line joining node1 and node2 is y=mx+c
    m = (node2.y-node1.y)/(node2.x-node1.x)
    c = node1.y-m*node1.x
    
    X = np.linspace(start = node1.x, stop = node2.x, num = 100)
    for x_ in X:
        y_ = m*x_+c
        
        x_, y_ = int(x_), int(y_)
        if(workspace[y_,x_]==0):
            return True
    return False
        

In [6]:
def dist(node1, node2):
    return math.sqrt((node1.x-node2.x)**2+(node1.y-node2.y)**2)

In [7]:
def generate_random_point():
    xrand = random.uniform(1,1499)
    yrand = random.uniform(1,1499)
    
    temp = node(xrand, yrand)
    if(temp.isObstacle() or not(temp.isValid())):
        return generate_random_point()
    else: return temp

In [8]:
def generate_nearest_point():
    
    # Generate random point in workspace    
    qrand = generate_random_point()
    
    # Find the nearest point to qrand in the graph: qnear
    firstKey = list(graph)[0]
    qnear = graph[firstKey]

    for q in graph.values():
        if(dist(qnear, qrand)>dist(q,qrand)):
            qnear = copy.copy(q)
    
    # Find qnew, which satisfies threshold distance
    if(dist(qnear, qrand)>thresh):
        theta = math.atan2((qrand.y-qnear.y),(qrand.x-qnear.y))
        x = thresh*math.cos(theta)+qnear.x
        y = thresh*math.sin(theta)+qnear.y
        qnew = node(x,y)
    else:
        qnew = copy.copy(qnear)
    
    return qnew

In [9]:
def plot_line(node1, node2):
    # TODO
    x_ = [node1.x, node2.x]
    y_ = [node1.y, node2.y]
    ax.plot(x_,y_, color='blue', linewidth = 0.5)


In [10]:
def function1(currentNode, qnew):
    
    if(isCollision(currentNode, qnew)):
        qnew = generate_nearest_point()
        function1(currentNode, qnew)
        return;
    else:
        plot_line(currentNode, qnew)
        qnew.set_parent(currentNode.x, currentNode.y)        
        graph.append(qnew)
        return;
        

In [11]:
c = 0

startingNode = node(startX, startY)
goalNode = node(goalX, goalY)

graph = []
graph.append(startingNode)
while c<numIterations:

    node_ = graph[0]
    qnew = generate_nearest_point()
    
    if(dist(qnew, goalNode)<= range):
        print('Destination Found!')
        goalNode.set_parent(qnew.x, qnew.y)
        path = retrace_path(goalNode)
    
    for p in nodes:
        if(dis(p, qnew)<dist(node_,qnew)):
            node_ = copy.copy(p)
            
    function1(node_, qnew) 

    if(not(qnew.isObstacle())):
        openList[(qnew.x, qnew.y)] = copy.copy(qnew)
        queue.append((qnew.x, qnew.y))
    del openList[(qnew.x, qnew.y)]

    c=c+1

KeyError: (960, 280)